# PydanticAI -- Agents, Tools & Chains

In the previous notebooks, we called the LLM and parsed JSON manually.
That works, but it's fragile -- the LLM might return broken JSON, wrong field names, or extra text.

**PydanticAI** fixes this. You define a **template** (what the output should look like),
and PydanticAI guarantees the LLM fills it in correctly. If it doesn't, it retries automatically.

Think of it like a form: you design the form, the LLM fills it in, PydanticAI checks every field.

In [ ]:
# !pip install pydantic-ai-slim[google]

In [ ]:
import os
os.environ["GOOGLE_API_KEY"] = "YOUR_KEY_HERE"

---
## 1. Your First Agent

An **Agent** is a wrapper around the LLM. You create it once, then ask it things.

```python
agent = Agent("model-name")      # create the agent
result = agent.run_sync("...")   # ask it something
result.output                     # the answer
```

That's it. Three lines.

In [ ]:
from pydantic_ai import Agent

agent = Agent("google-gla:gemini-2.0-flash")
result = agent.run_sync("What is IFC? One sentence.")
print(result.output)

---
## 2. Structured Output (the whole point)

Remember manually parsing JSON with `json.loads()`? No more.

We define a **model** -- a Python class that describes the shape of the data we want.
Each field has a name and a type. PydanticAI forces the LLM to return exactly this shape.

```python
class DoorCheck(BaseModel):     # <-- this is the "form"
    door_name: str              # text
    width_mm: float             # number
    passed: bool                # true or false
```

Then pass it as `output_type` -- the agent now **must** return a filled-in DoorCheck.

In [ ]:
from pydantic import BaseModel

class DoorCheck(BaseModel):
    door_name: str
    width_mm: float
    min_required_mm: float
    passed: bool

agent = Agent("google-gla:gemini-2.0-flash", output_type=DoorCheck)
result = agent.run_sync("Door A12 is 780mm wide. Minimum is 800mm.")

print(result.output)              # the full object
print(f"Passed? {result.output.passed}")   # access fields directly

No `json.loads()`, no stripping markdown fences, no hoping the LLM got the field names right.
It just works.

---
## 3. Chain -- One Agent Feeds Into Another

A **chain** = the output of Agent A becomes the input for Agent B.

Like an assembly line: one worker extracts the data, the next worker makes the decision.

```
"Door is 780mm, min 800mm"  -->  Agent A (extract)  -->  {rule, actual}  -->  Agent B (decide)  -->  true/false
```

In [ ]:
class Extract(BaseModel):
    rule: str
    actual: str

extractor = Agent("google-gla:gemini-2.0-flash", output_type=Extract)
checker   = Agent("google-gla:gemini-2.0-flash", output_type=bool)

# Step 1: extract the facts
a = extractor.run_sync("Door A12 width is 780mm; min required is 800mm.")
print(f"Extracted: rule='{a.output.rule}', actual='{a.output.actual}'")

# Step 2: feed into the next agent
b = checker.run_sync(f"Does it pass? rule={a.output.rule} actual={a.output.actual}")
print(f"Passed: {b.output}")

---
## 4. Tools -- Give the Agent Superpowers

A **tool** is a Python function that the agent can call whenever it needs to.
The agent reads your question, decides *"I need to use this tool"*, calls it, and uses the result.

You register a tool using the **`@agent.tool` decorator**.

### What's a decorator?

If you haven't seen `@something` before -- it's just a tag you put above a function.
It means: *"hey agent, this function exists, you can use it."*

```python
@orchestrator.tool                          # <-- "register this as a tool"
def check_door(ctx: RunContext[None], ...)   # <-- the function
```

The `ctx: RunContext[None]` part is just required syntax -- ignore it for now, it's always there.

The agent reads the function name and its **docstring** (the text in triple quotes)
to understand what the tool does. So write a clear docstring!

In [ ]:
from pydantic_ai import RunContext

orchestrator = Agent("google-gla:gemini-2.0-flash", output_type=str)


@orchestrator.tool
def check_door_width(ctx: RunContext[None], width_mm: float, min_mm: float) -> str:
    """Check if a door width meets the minimum requirement."""
    passed = width_mm >= min_mm
    return f"passed={passed} (width={width_mm}, min={min_mm})"


result = orchestrator.run_sync("Is a 780mm door OK if minimum is 800mm?")
print(result.output)

What happened behind the scenes:
1. The agent read your question
2. It saw it has a tool called `check_door_width`
3. It called `check_door_width(width_mm=780, min_mm=800)`
4. It got `"passed=False ..."` back
5. It wrote a human-readable answer using that result

**You write the tools. The agent decides when to use them.**

---
## 5. Putting It Together -- Orchestrator with a Chain as a Tool

Now the big one: an agent that can call **other agents** as tools.

This is how your compliance checker will work:
- The **orchestrator** receives a question
- It calls specialized tools (door check, staircase check, etc.)
- Each tool can itself use agents internally
- The orchestrator combines everything into a final answer

In [ ]:
smart_agent = Agent("google-gla:gemini-2.0-flash", output_type=str)


@smart_agent.tool
def run_compliance_check(ctx: RunContext[None], description: str) -> str:
    """Run a full compliance check on a building element description."""
    # This tool internally uses the chain from section 3!
    a = extractor.run_sync(description).output
    ok = checker.run_sync(
        f"Does it pass? rule={a.rule} actual={a.actual}"
    ).output
    return f"passed={ok}, rule='{a.rule}', actual='{a.actual}'"


result = smart_agent.run_sync(
    "Check these two:\n"
    "1. Door D1 is 850mm wide, min is 800mm\n"
    "2. Door D2 is 720mm wide, min is 800mm"
)
print(result.output)

---
## Recap

| Concept | What it is | One-liner |
|---|---|---|
| **Agent** | Wrapper around the LLM | `Agent("model")` |
| **output_type** | Forces the LLM to return a specific shape | `output_type=MyModel` |
| **Chain** | Output of agent A feeds into agent B | Just call two agents in sequence |
| **Tool** | A function the agent can call | `@agent.tool` decorator |
| **Orchestrator** | An agent whose tools call other agents | Nesting agents inside tools |

**This is the pattern for your compliance checker app.** Each check is a tool, the orchestrator calls them all.